In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Linear transformations for Query (Q), Key (K), Value (V), and output (O)
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores using Q, K, and scaling by the square root of d_k
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Apply the softmax function to get attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Weighted sum of values (V) using attention probabilities
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations to Q, K, and V and then split into heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        # Perform scaled dot-product attention for each head
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        # Combine the output from multiple heads and apply the output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
# The class initializes with two linear transformation layers and a ReLU activation function.
# The forward method applies these transformations and activation function sequentially to compute the output.
# This process enables the model to consider the position of input elements while making predictions.

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
# Positional Encoding is used to inject the position information of each token in the input sequence.

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        # Create a position tensor with values from 0 to max_seq_length - 1
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        # Calculate sine and cosine components of the positional encoding
        # The class calculates sine and cosine values for even and odd indices, respectively, based on the scaling factor div_term.
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [ ]:
# An Encoder layer consists of a Multi-Head Attention layer, a Position-wise Feed-Forward layer, and two Layer Normalization layers.

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        # create a self-attention mechanism using MultiHeadAttention
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        # create a position-wise feed-forward network
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        # Regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [ ]:
# A Decoder layer consists of two Multi-Head Attention layers, a Position-wise Feed-Forward layer, and three Layer Normalization layers.
# The DecoderLayer initializes with input parameters and components such as MultiHeadAttention modules for masked self-attention and cross-attention,
# a PositionWiseFeedForward module, three layer normalization modules, and a dropout layer.

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        # cross-attention mechanism to the source sequence
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        # normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        # drop out for regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        # Calculate the masked self-attention output and add it to the input tensor, followed by dropout and layer normalization.
        x = self.norm1(x + self.dropout(attn_output))
        # cross-attention output between the decoder and encoder outputs
        # add it to the normalized masked self-attention output
        # followed by dropout and layer normalization.
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        #  position-wise feed-forward output
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        # Embeddings for the encoder and decoder
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
         # positional encodings for both encoder and decoder
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        # put them together
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        # Generate source and target masks
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [ ]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.687689781188965
Epoch: 2, Loss: 8.552364349365234
Epoch: 3, Loss: 8.478200912475586
Epoch: 4, Loss: 8.42687702178955
Epoch: 5, Loss: 8.369094848632812
Epoch: 6, Loss: 8.304137229919434
Epoch: 7, Loss: 8.224376678466797
Epoch: 8, Loss: 8.141823768615723
Epoch: 9, Loss: 8.062759399414062
Epoch: 10, Loss: 7.985346794128418
Epoch: 11, Loss: 7.899988651275635
Epoch: 12, Loss: 7.829467296600342
Epoch: 13, Loss: 7.739632606506348
Epoch: 14, Loss: 7.65571403503418
Epoch: 15, Loss: 7.5759735107421875
Epoch: 16, Loss: 7.492513179779053
Epoch: 17, Loss: 7.408785343170166
Epoch: 18, Loss: 7.323356628417969
Epoch: 19, Loss: 7.247158050537109
Epoch: 20, Loss: 7.158466339111328
Epoch: 21, Loss: 7.080060958862305
Epoch: 22, Loss: 7.006396770477295
Epoch: 23, Loss: 6.9300312995910645
Epoch: 24, Loss: 6.853365898132324
Epoch: 25, Loss: 6.772275924682617
Epoch: 26, Loss: 6.711093902587891
Epoch: 27, Loss: 6.6305975914001465
Epoch: 28, Loss: 6.558922290802002
Epoch: 29, Loss: 6.493185520